In [1]:
from keras import layers
from keras import models

model = models.Sequential()

model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)))

model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Conv2D(64, (3, 3), activation='relu'))

model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Conv2D(64, (3, 3), activation='relu'))

2023-10-22 13:34:45.041352: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-10-22 13:34:45.041406: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-10-22 13:34:45.041444: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-10-22 13:34:45.049015: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-10-22 13:34:46.063438: W tensorflow/compiler/

In [2]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 26, 26, 32)        320       
                                                                 
 max_pooling2d (MaxPooling2  (None, 13, 13, 32)        0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 11, 11, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 5, 5, 64)          0         
 g2D)                                                            
                                                                 
 conv2d_2 (Conv2D)           (None, 3, 3, 64)          36928     
                                                                 
Total params: 55744 (217.75 KB)
Trainable params: 55744 

In [3]:
model.add(layers.Flatten())

# 3 * 3 * 64 = 576
model.add(layers.Dense(64, activation='relu'))

model.add(layers.Dense(10, activation='softmax'))

In [4]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 26, 26, 32)        320       
                                                                 
 max_pooling2d (MaxPooling2  (None, 13, 13, 32)        0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 11, 11, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 5, 5, 64)          0         
 g2D)                                                            
                                                                 
 conv2d_2 (Conv2D)           (None, 3, 3, 64)          36928     
                                                                 
 flatten (Flatten)           (None, 576)               0

In [5]:
from keras.datasets import mnist
from keras.utils import to_categorical

(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

print(f"{train_images.shape=}")
print(f"{train_labels.shape=}")
print(f"{test_images.shape=}")
print(f"{test_labels.shape=}")

train_images.shape=(60000, 28, 28)
train_labels.shape=(60000,)
test_images.shape=(10000, 28, 28)
test_labels.shape=(10000,)


In [6]:
train_images = train_images.reshape((60000, 28, 28, 1))
train_images = train_images.astype('float32') / 255

test_images = test_images.reshape((10000, 28, 28, 1))
test_images = test_images.astype('float32') / 255

print(f"{train_images.shape=}")
print(f"{test_images.shape=}")

train_images.shape=(60000, 28, 28, 1)
test_images.shape=(10000, 28, 28, 1)


In [7]:
train_labels = to_categorical(train_labels)
test_labels = to_categorical(test_labels)

print(f"{train_labels.shape=}")
print(f"{test_labels.shape=}")

train_labels.shape=(60000, 10)
test_labels.shape=(10000, 10)


In [8]:
from keras import callbacks

logdir = "../my_logdir"
best_model = logdir + "/my_best_model.keras"
my_callbacks = [
    callbacks.ModelCheckpoint(filepath=best_model, save_best_only=True),
    callbacks.TensorBoard(log_dir=logdir)
]
model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

history = model.fit(
    train_images, 
    train_labels, 
    epochs=5, 
    batch_size=64,
    callbacks=my_callbacks
)

Epoch 1/5
938/938 [==============================] - 15s 15ms/step - loss: 0.1739 - accuracy: 0.9449
Epoch 2/5
938/938 [==============================] - 14s 15ms/step - loss: 0.0473 - accuracy: 0.9853
Epoch 3/5
938/938 [==============================] - 14s 15ms/step - loss: 0.0334 - accuracy: 0.9897
Epoch 4/5
938/938 [==============================] - 14s 15ms/step - loss: 0.0252 - accuracy: 0.9920
Epoch 5/5
938/938 [==============================] - 14s 15ms/step - loss: 0.0190 - accuracy: 0.9937


In [9]:
print(history.history.items())
for k, v in history.history.items():
    print(f"{k}: {v}")

dict_items([('loss', [0.1738741099834442, 0.047282151877880096, 0.03340259939432144, 0.02519937977194786, 0.018981611356139183]), ('accuracy', [0.9448500275611877, 0.9853000044822693, 0.9896500110626221, 0.9919833540916443, 0.9937166571617126])])
loss: [0.1738741099834442, 0.047282151877880096, 0.03340259939432144, 0.02519937977194786, 0.018981611356139183]
accuracy: [0.9448500275611877, 0.9853000044822693, 0.9896500110626221, 0.9919833540916443, 0.9937166571617126]


In [10]:
result= model.evaluate(test_images, test_labels)


313/313 [==============================] - 1s 3ms/step - loss: 0.0372 - accuracy: 0.9903


In [11]:
print(model.metrics_names)

['loss', 'accuracy']


In [12]:
for i, value in enumerate(result):
    print(f"{model.metrics_names[i]}: {value}")

loss: 0.03724966570734978
accuracy: 0.9902999997138977
